# Определение стоимости автомобилей

Для сервиса по продаже автомобилей необходимо построить модель, определяющую стоимость авто.

В нашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей.

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.
<br>
<br>

**Описание данных**<br>
Признаки
- DateCrawled — дата скачивания анкеты из базы
- VehicleType — тип автомобильного кузова
- RegistrationYear — год регистрации автомобиля
- Gearbox — тип коробки передач
- Power — мощность (л. с.)
- Model — модель автомобиля
- Kilometer — пробег (км)
- RegistrationMonth — месяц регистрации автомобиля
- FuelType — тип топлива
- Brand — марка автомобиля
- NotRepaired — была машина в ремонте или нет
- DateCreated — дата создания анкеты
- NumberOfPictures — количество фотографий автомобиля
- PostalCode — почтовый индекс владельца анкеты (пользователя)
- LastSeen — дата последней активности пользователя

Целевой признак
- Price — цена (евро)

## Подготовка данных

Загрузим библиотеки и откроем файл с данными

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, make_scorer
from lightgbm import LGBMRegressor


data = pd.read_csv('/datasets/autos.csv')
display(data.head(10))
display(data.describe())
display(data.info())
data.corr()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21
5,2016-04-04 17:36:23,650,sedan,1995,manual,102,3er,150000,10,petrol,bmw,yes,2016-04-04 00:00:00,0,33775,2016-04-06 19:17:07
6,2016-04-01 20:48:51,2200,convertible,2004,manual,109,2_reihe,150000,8,petrol,peugeot,no,2016-04-01 00:00:00,0,67112,2016-04-05 18:18:39
7,2016-03-21 18:54:38,0,sedan,1980,manual,50,other,40000,7,petrol,volkswagen,no,2016-03-21 00:00:00,0,19348,2016-03-25 16:47:58
8,2016-04-04 23:42:13,14500,bus,2014,manual,125,c_max,30000,8,petrol,ford,NaN,2016-04-04 00:00:00,0,94505,2016-04-04 23:42:13
9,2016-03-17 10:53:50,999,small,1998,manual,101,golf,150000,0,NaN,volkswagen,NaN,2016-03-17 00:00:00,0,27472,2016-03-31 17:17:06


,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
DateCrawled          354369 non-null object
Price                354369 non-null int64
VehicleType          316879 non-null object
RegistrationYear     354369 non-null int64
Gearbox              334536 non-null object
Power                354369 non-null int64
Model                334664 non-null object
Kilometer            354369 non-null int64
RegistrationMonth    354369 non-null int64
FuelType             321474 non-null object
Brand                354369 non-null object
NotRepaired          283215 non-null object
DateCreated          354369 non-null object
NumberOfPictures     354369 non-null int64
PostalCode           354369 non-null int64
LastSeen             354369 non-null object
dtypes: int64(7), object(9)
memory usage: 43.3+ MB


None

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
Price,1.000000,0.026916,0.158872,-0.333199,0.110581,NaN,0.076055
RegistrationYear,0.026916,1.000000,-0.000828,-0.053447,-0.011619,NaN,-0.003459
Power,0.158872,-0.000828,1.000000,0.024002,0.043380,NaN,0.021665
Kilometer,-0.333199,-0.053447,0.024002,1.000000,0.009571,NaN,-0.007698
RegistrationMonth,0.110581,-0.011619,0.043380,0.009571,1.000000,NaN,0.013995
NumberOfPictures,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PostalCode,0.076055,-0.003459,0.021665,-0.007698,0.013995,NaN,1.000000


Наблюдаем множественные пропуски в некоторых столбцах, а также отсутствие корреляционных признаков.

Проверим данные на наличие дубликатов.

In [2]:
data.duplicated().sum()

4

Удалим их.

In [3]:
data = data.drop_duplicates()

Теперь посмотрим на количество пропусков

In [4]:
print(data.isna().sum())

DateCrawled              0
Price                    0
VehicleType          37490
RegistrationYear         0
Gearbox              19833
Power                    0
Model                19705
Kilometer                0
RegistrationMonth        0
FuelType             32895
Brand                    0
NotRepaired          71154
DateCreated              0
NumberOfPictures         0
PostalCode               0
LastSeen                 0
dtype: int64


Для поиска ошибок посмотрим на уникальные значения в каждом столбце

In [5]:
for i in data.columns:
    print(i)
    print(data[i].value_counts())
    print(data[i].unique())
    print()

DateCrawled
2016-03-24 14:49:47    7
2016-03-19 21:49:56    6
2016-03-26 22:57:31    6
2016-03-05 14:25:23    5
2016-03-05 14:44:30    5
                      ..
2016-03-13 00:57:18    1
2016-03-22 17:52:11    1
2016-03-25 14:53:40    1
2016-04-02 16:57:00    1
2016-04-03 22:47:46    1
Name: DateCrawled, Length: 271174, dtype: int64
['2016-03-24 11:52:17' '2016-03-24 10:58:45' '2016-03-14 12:52:21' ...
 '2016-03-21 09:50:58' '2016-03-14 17:48:27' '2016-03-19 18:57:12']

Price
0        10772
500       5670
1500      5394
1000      4648
1200      4594
         ...  
13440        1
1414         1
8069         1
10370        1
384          1
Name: Price, Length: 3731, dtype: int64
[  480 18300  9800 ... 12395 18429 10985]

VehicleType
sedan          91457
small          79830
wagon          65165
bus            28775
convertible    20203
coupe          16161
suv            11996
other           3288
Name: VehicleType, dtype: int64
[nan 'coupe' 'suv' 'small' 'sedan' 'convertible' 'bus' 'wag

Что наблюдаем:
- В столбце "Price" есть нулевые значения - аномалия. Авто на продажу идет бесплатно? Возможно в следствии нежелания владельцев тратить средства на утилизацию машины. Пропускаем.
- 'VehicleType'. Более 37тыс. пропущенных значений. Заменим пропуски группой "unknown", т.к. нет возможности точно определить к какому типу кузова относится та или иная модель.
- Столбец с годом регистрации тоже отличился неправильными значениями. Можем видеть 3500 и даже 9229 года.. Несопоставимо с реальностью. Ограничим возможный разброс от 1900 года и по наше время (т.е. по 2021 год).
- "Gearbox". Около 5% пропущенных значений. Трудно предположить к чему отнести пропуск. Поступим также, как и с 'VehicleType'.
- Мощность. Нулевые и сверхбольшие значения. Нулевая мощность может объясняться машиной не на ходу, в то время, как сверхбольшие значения стоит удалить вовсе.
- Модель. Около 5% пропущенных значений. Заполним их по аналогии с "VehicleType" и "Gearbox".
- Тип топлива. Заменим пропуски на "unknown".
- Ремонт. Отсутствие пометки о ремонте автомобиля можно засчитать за его целость, т.е. "no".

Заменим пропущенные значения в 'vehicle_type'

In [6]:
data['VehicleType'] = data['VehicleType'].fillna(value = 'unknown')

Поставим ограничения по году регистрации (1900;2021)

In [7]:
data = data.query('(RegistrationYear >= 1900) and (RegistrationYear <= 2021)').reset_index(drop=True)

Заменим пропущенные значения в колонке "Gearbox" на 'unknown'

In [8]:
data['Gearbox'] = data['Gearbox'].fillna(value = 'unknown')

Ограничим мощность.<br>
Самый мощный автомобиль на данный момент: Aspark Owl. У него 2012 л.с. <br>
Это и будет наше ограничение.

In [9]:
data = data.query('Power < 2012')

Заполним пропуски в столбце "Model" значением "unknown".

In [10]:
data['Model'] = data['Model'].fillna(value = 'unknown')

Поступим также со столбцом "FuelType"

In [11]:
data['FuelType'] = data['FuelType'].fillna(value = 'unknown')

Поменяем пропуски на значение "no" (объяснение выше).

In [12]:
data['NotRepaired'] = data['NotRepaired'].fillna(value = 'no')

Проверим датасет на наличие пропусков.

In [13]:
data.isna().sum()

DateCrawled          0
Price                0
VehicleType          0
RegistrationYear     0
Gearbox              0
Power                0
Model                0
Kilometer            0
RegistrationMonth    0
FuelType             0
Brand                0
NotRepaired          0
DateCreated          0
NumberOfPictures     0
PostalCode           0
LastSeen             0
dtype: int64

Отлично!<br>
Однако, у нас есть признаки, которые маловероятно могут повлиять на предсказание цены автомобиля. Среди них: 
- DateCrawled, 
- RegistrationMonth,
- DateCreated,
- NumberOfPictures,
- PostalCode,
- LastSeen

Удалим их.

In [14]:
data = data.drop(['DateCrawled', 
                  'RegistrationMonth', 
                  'DateCreated',
                  'NumberOfPictures',
                  'PostalCode',
                  'LastSeen'], axis = 1)

**Теперь перейдем к подготовке данных для машинного обучения.**

Для обучения моделей необходимо преобразовать категориальные признаки в численные.<br>

Для столбцов с большим числом значений (Model, Brand) будем использовать технику порядкового кодирования, чтобы меньше воздействовать на размер выборки.<br>

Для столбцов с маленьким количеством значений (VehicleType, Gearbox, Fuel_Type, NotRepaired) будем использовать технику OHE.

Для удобства разделим признаки.

In [15]:
numeric = ['Price', 'RegistrationYear', 'Power', 'Kilometer']
ordinal = ['Model', 'Brand']
ohe = ['VehicleType', 'Gearbox', 'FuelType', 'NotRepaired']

Выполним порядковое кодирование.

In [16]:
encoder = OrdinalEncoder()
data_ordinal = pd.DataFrame(encoder.fit_transform(data.loc[:, ordinal]), columns = ordinal)

Выполним OHE.

In [17]:
data_OHE = pd.get_dummies(data.loc[:, ohe], drop_first = True)

Объединим датафреймы.

In [18]:
data_01 = data[numeric].join([data_ordinal, data_OHE])
data_01.dropna(subset = ['Model', 'Brand'], inplace = True) 
data_01.head(10)

,Price,RegistrationYear,Power,Kilometer,Model,Brand,VehicleType_convertible,VehicleType_coupe,VehicleType_other,VehicleType_sedan,...,Gearbox_manual,Gearbox_unknown,FuelType_electric,FuelType_gasoline,FuelType_hybrid,FuelType_lpg,FuelType_other,FuelType_petrol,FuelType_unknown,NotRepaired_yes
0,480.0,1993.0,0.0,150000.0,116.0,38.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,18300.0,2011.0,190.0,125000.0,228.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,9800.0,2004.0,163.0,125000.0,117.0,14.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1500.0,2001.0,75.0,150000.0,116.0,38.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,3600.0,2008.0,69.0,90000.0,101.0,31.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5,650.0,1995.0,102.0,150000.0,11.0,2.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
6,2200.0,2004.0,109.0,150000.0,8.0,25.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7,0.0,1980.0,50.0,40000.0,166.0,38.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
8,14500.0,2014.0,125.0,30000.0,60.0,10.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9,999.0,1998.0,101.0,150000.0,116.0,38.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


Теперь разделим наши обработанные данные на обучающую и валидационную выборки.

In [19]:
features = data_01.drop('Price', axis=1)
target = data_01['Price']

features_train, features_valid, target_train, target_valid = train_test_split(features, target,  
                                            test_size=0.4, random_state=12345)

features_valid, features_test, target_valid, target_test = train_test_split(features_valid, target_valid,  
                                            test_size=0.5, random_state=12345)

display(features_train.shape)
display(features_valid.shape)
features_test.shape

(212379, 23)

(70793, 23)

(70794, 23)

Промасшатибруем количественные данные.

In [20]:
scaler = StandardScaler()
kol = ['RegistrationYear', 'Power', 'Kilometer']
scaler.fit(features_train[kol])
features_train[kol] = scaler.transform(features_train[kol])
features_valid[kol] = scaler.transform(features_valid[kol])
features_test[kol] = scaler.transform(features_test[kol])

features_train = features_train.dropna()
features_valid = features_valid.dropna()
features_test = features_test.dropna()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:3498: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.loc._setitem_with_indexer((slice(None), indexer), value)
/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:3469: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guid

Данные готовы. Можем приступать к обучению.

## Обучение моделей

Создадим scorer для оценки моделей.<br>
В нашем случае им будет выступать RMSE (root mean squared error)

In [21]:
def rmse(target, prediction):
    rmse =  (np.sum(prediction - target)**2 / len(prediction))**0.5
    return rmse

rmse_scorer = make_scorer(rmse, greater_is_better = False)

### Дерево решений

Обучим дерево решений при помощи GridSearchCV, подобрав лучшие гиперпараметры.

In [22]:
%%time

tree = {'max_depth': range(1, 12)}
model_tree = DecisionTreeRegressor(random_state = 12345)
grid_tree = GridSearchCV(model_tree, tree, cv = 5, scoring = rmse_scorer)
grid_tree.fit(features_train, target_train)
print('RMSE для дерева решений:', abs(grid_tree.best_score_))
print()
print(grid_tree.best_estimator_)

RMSE для дерева решений: 1925.806572859328

DecisionTreeRegressor(criterion='mse', max_depth=7, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=12345, splitter='best')
CPU times: user 24.9 s, sys: 35.9 ms, total: 24.9 s
Wall time: 25 s


Проверим валидационные данные с нашей оптимальной глубиной.

In [23]:
model_tree = DecisionTreeRegressor(random_state = 12345, max_depth = 11)
model_tree.fit(features_train, target_train)
prediction = model_tree.predict(features_valid)
print('RMSE для дерева решений:', rmse(target_valid, prediction))

RMSE для дерева решений: 1421.8323083851471


Метрика выдает приблизительно равные величины, это хорошо. Значит модель не переобучилась.<br>
Теперь предскажем целевой признак для тестового набора данных.

In [24]:
prediction = model_tree.predict(features_test)
print('RMSE для дерева решений:', rmse(target_test, prediction))

RMSE для дерева решений: 1164.736195003713


RMSE = 1164, нормально ли это при изначальных 1925 ? Видимо словили переобучение. 

### Случайный лес

Случайный лес обучим прямым способом, посмотрев разные гиперпараметры глубины леса и количества деревьев. Иначе время обучения может затянуться надолго. 

In [25]:
%%time

def m_forest(max_depth, estimat):
    model_forest = RandomForestRegressor(random_state = 12345, max_depth = max_depth, n_estimators = estimat)
    model_forest.fit(features_train, target_train)
    prediction = model_forest.predict(features_valid)
    rmse = mean_squared_error(target_valid, prediction)**0.5
    print('Глубина:', max_depth, ', Количество деревьев:', estimat, ", RMSE:", rmse)
    print()
    
m_forest(10, 10)
m_forest(10, 12)
m_forest(10, 15)
m_forest(15, 10)
m_forest(15, 12)
m_forest(15, 15)

m_forest(15, 17)
m_forest(15, 20)
m_forest(15, 25)
m_forest(15, 40)
m_forest(15, 45)

Глубина: 10 , Количество деревьев: 10 , RMSE: 2118.9070494426355

Глубина: 10 , Количество деревьев: 12 , RMSE: 2116.238602262925

Глубина: 10 , Количество деревьев: 15 , RMSE: 2114.512722409185

Глубина: 15 , Количество деревьев: 10 , RMSE: 2022.6377596524885

Глубина: 15 , Количество деревьев: 12 , RMSE: 2018.2219089258085

Глубина: 15 , Количество деревьев: 15 , RMSE: 2013.652445619778

Глубина: 15 , Количество деревьев: 17 , RMSE: 2009.4733598022917

Глубина: 15 , Количество деревьев: 20 , RMSE: 2006.863457488394

Глубина: 15 , Количество деревьев: 25 , RMSE: 2002.3303440759005

Глубина: 15 , Количество деревьев: 40 , RMSE: 1998.3384467980463

Глубина: 15 , Количество деревьев: 45 , RMSE: 1997.436863506363

CPU times: user 2min 49s, sys: 115 ms, total: 2min 49s
Wall time: 2min 49s


Все же попробуем перебрать несколько деревьев через цикл с глубиной и количеством.

In [26]:
%%time

best_est = 0
best_depth = 0
best_model = None
best_rmse = 10000
for est in range(10, 40, 5):
    for depth in range(1, 5):
        model_forest = RandomForestRegressor(random_state = 12345, max_depth = depth, n_estimators = est)
        model_forest.fit(features_train, target_train)
        prediction = model_forest.predict(features_valid)
        rmse = mean_squared_error(target_valid, prediction)**0.5
        if rmse < best_rmse:
            best_rmse = rmse
            best_model = model_forest
            best_est = est
            best_depth = depth
            
print('Глубина:', best_depth, ', Количество деревьев:', best_est, ", RMSE:", best_rmse)

Глубина: 4 , Количество деревьев: 15 , RMSE: 2639.1518524194557
CPU times: user 1min 26s, sys: 47.7 ms, total: 1min 26s
Wall time: 1min 27s


Как видим, прогноз лучше удался при случайно выбранных гиперпараметрах (одним выводом выше ).<br>
Спрогнозируем значения для тестовых данных (при лучших значениях глубины и количества).

In [27]:
model_forest = RandomForestRegressor(random_state = 12345, max_depth = 15, n_estimators = 45)
model_forest.fit(features_train, target_train)
prediction = model_forest.predict(features_test)
rmse = mean_squared_error(target_test, prediction)**0.5
print('RMSE для случайного леса:', rmse)

RMSE для случайного леса: 2005.7521742982412


RMSE у случайного леса = 2006 на тестовых данных. 

### Линейная регрессия

В линейной регрессии попробуем кросс-валидацию с применением нашего scorer.

In [28]:
%%time

model_linear = LinearRegression()
cross_linear = cross_val_score(model_linear, features_train, target_train, cv = 5, scoring = rmse_scorer)
mean_rmse = abs(cross_linear).mean()
print('RMSE:', mean_rmse)
print()

RMSE: 3367.3170281781495

CPU times: user 1.59 s, sys: 1.13 s, total: 2.72 s
Wall time: 2.68 s


Проверим на валидационной выборке.

In [36]:
%%time

model_linear.fit(features_train, target_train)
prediction = model_linear.predict(features_valid)
rmse = mean_squared_error(target_valid, prediction)**0.5
print('RMSE для валидационной выборки на линейной регрессии:', rmse)
print()

RMSE для валидационной выборки на линейной регрессии: 3123.1756795667807

CPU times: user 355 ms, sys: 108 ms, total: 463 ms
Wall time: 419 ms


Результаты не вдохновляют.. Возможно ли доработать ?

Спрогнозируем значения для тестовой выборки и найдем ошибку.

In [30]:
%%time

prediction_test = model_linear.predict(features_test)
rmse = mean_squared_error(target_test, prediction_test)**0.5
print('RMSE для валидационной выборки на линейной регрессии:', rmse)
print()

RMSE для валидационной выборки на линейной регрессии: 3142.093224904493

CPU times: user 23.5 ms, sys: 162 µs, total: 23.6 ms
Wall time: 5.9 ms


### LightGBM

Посмотрим на работу LightGBM

In [31]:
%%time

model_lgb = LGBMRegressor(max_depth = 10, n_estimators = 90, num_leaves = 20)
model_lgb.fit(features_train, target_train, feature_name = 'auto', categorical_feature = 'auto', verbose = 10)
prediction = model_lgb.predict(features_valid)
rmse = mean_squared_error(target_valid, prediction)**0.5
print('RMSE на LightGBM:', rmse)

RMSE на LightGBM: 2053.4207042000976
CPU times: user 1min 11s, sys: 432 ms, total: 1min 11s
Wall time: 1min 12s


Теперь применим GridSearchCV для построения модели градиентного бустинга.

In [32]:
%%time

model_lgb_01 = LGBMRegressor()
param = {'n_estimators': [10,50,100],
        'num_leaves': [10,20,30]}

grid_gbm = GridSearchCV(model_lgb_01, param, cv=3, scoring = rmse_scorer, verbose = 5)
grid_gbm.fit(features_train, target_train)
rmse = abs(grid_gbm.best_score_)
print('RMSE у LGBMRegressor:', rmse)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] n_estimators=10, num_leaves=10 ..................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .. n_estimators=10, num_leaves=10, score=-2253.876, total=   1.7s
[CV] n_estimators=10, num_leaves=10 ..................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.8s remaining:    0.0s


[CV] .. n_estimators=10, num_leaves=10, score=-1285.809, total=   2.4s
[CV] n_estimators=10, num_leaves=10 ..................................


...

[CV] .. n_estimators=10, num_leaves=20, score=-2258.406, total=  22.8s
[CV] n_estimators=10, num_leaves=20 ..................................


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:   29.3s remaining:    0.0s


[CV] .. n_estimators=10, num_leaves=20, score=-1642.907, total=  10.0s
[CV] n_estimators=10, num_leaves=20 ..................................
[CV] .. n_estimators=10, num_leaves=20, score=-3710.464, total=   6.8s
[CV] n_estimators=10, num_leaves=30 ..................................
[CV] .. n_estimators=10, num_leaves=30, score=-1691.018, total=   9.7s
[CV] n_estimators=10, num_leaves=30 ..................................
[CV] .. n_estimators=10, num_leaves=30, score=-1693.387, total=  10.9s
[CV] n_estimators=10, num_leaves=30 ..................................
[CV] .. n_estimators=10, num_leaves=30, score=-3649.225, total=   2.1s
[CV] n_estimators=50, num_leaves=10 ..................................
[CV] ... n_estimators=50, num_leaves=10, score=-489.645, total=  18.6s
[CV] n_estimators=50, num_leaves=10 ..................................
[CV] .. n_estimators=50, num_leaves=10, score=-2398.495, total=  15.6s
[CV] n_estimators=50, num_leaves=10 ..................................
[CV] .

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:  3.5min finished


RMSE у LGBMRegressor: 1784.98820871295
CPU times: user 3min 30s, sys: 2.09 s, total: 3min 32s
Wall time: 3min 34s


Довольно неплохой показатель за такое время работы..

Проверим валидационные данные.

In [33]:
prediction = grid_gbm.predict(features_valid)
rmse = mean_squared_error(target_valid, prediction)**0.5
print('RMSE у LGBMRegressor:', rmse)

RMSE у LGBMRegressor: 2102.1757900570306


Отработаем на тестовых.

In [34]:
prediction = grid_gbm.predict(features_test)
rmse = mean_squared_error(target_test, prediction)**0.5
print('RMSE у LGBMRegressor:', rmse)

RMSE у LGBMRegressor: 2115.5811146200467


## Анализ моделей

Показатели времени обучения и качества предсказания представлены выше (уже в обучении моделей).

По ним можно сказать, что самая оптимальная по времени, и с самой лучшей точностью является модель случайного леса. Подавал надежды  LGBMRegressor, однако справился с задачей хуже + время обучения выше.<br>
Средняя цена машины:

In [35]:
data_01['Price'].mean()

4418.093376199974

Но даже при такой средней цене машины, отклонение в 2 тыс. евро - слишком много.. Это около 45% от средней стоимости!!!<br>
Результат нельзя считать позитивным.